<a href="https://colab.research.google.com/github/Tanveer132/ML-Practices-End-to-End/blob/main/Drug_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Drug Recommendation -

**Aproach -** 
    
    1.To use data of different medicines and their manufacturers.
    2.Build rules to suggest different brands for single medicines.

In [1]:
#Importing necessary libraries
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

**dataset** - drugs main list

This data provides information about different brands / manufacturers of medicine.

In [2]:
df1 = pd.read_csv("/content/Drugs-main list.csv")
df1.head()

,Drug_name,US_brands
0,Antipyrine And Benzocaine (Otic Route),"['AB Otic', 'Aurodex', 'Auroto', 'Benzotic', '..."
1,Erythromycin (Topical Route),"['ATS', 'Akne-Mycin', 'Emcin', 'Emgel', 'Ery',..."
2,Pyrethrum Extract And Piperonyl Butoxide (Topi...,"['A200 Maximum Strength', 'A200 Time-Tested Fo..."
3,Pyrethrum Extract And Piperonyl Butoxide (Topi...,"['A200 Maximum Strength', 'A200 Time-Tested Fo..."
4,Laxative (Oral Route),"['A-3 Revised', 'Alocass', 'Amitiza', 'Benefib..."


Data Preprocessing and bringing to suitable format.

In [3]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5820 entries, 0 to 5819
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Drug_name  5820 non-null   object
 1   US_brands  5795 non-null   object
dtypes: object(2)
memory usage: 91.1+ KB


In [4]:
df1 = df1.dropna()
df1 = df1.reset_index(drop=True)
df1.head()

,Drug_name,US_brands
0,Antipyrine And Benzocaine (Otic Route),"['AB Otic', 'Aurodex', 'Auroto', 'Benzotic', '..."
1,Erythromycin (Topical Route),"['ATS', 'Akne-Mycin', 'Emcin', 'Emgel', 'Ery',..."
2,Pyrethrum Extract And Piperonyl Butoxide (Topi...,"['A200 Maximum Strength', 'A200 Time-Tested Fo..."
3,Pyrethrum Extract And Piperonyl Butoxide (Topi...,"['A200 Maximum Strength', 'A200 Time-Tested Fo..."
4,Laxative (Oral Route),"['A-3 Revised', 'Alocass', 'Amitiza', 'Benefib..."


In [5]:
df1['US_brands'][0][0]
#Here each row contains string which we need to convert in list of brands.

'['

In [6]:
#we get list of brands for 1st medicine
df1['US_brands'][0].replace("[","").replace("]","").replace("'","").split(",")

['AB Otic', ' Aurodex', ' Auroto', ' Benzotic', ' Dec-Agesic AB', ' Dolotic']

In [7]:
type(df1['US_brands'][0])

str

In [8]:
for i in range(df1.shape[0]):
    if type(df1['US_brands'][i])==str:
        df1['US_brands'][i]=df1['US_brands'][i].replace("[","").replace("]","").strip().replace("'","").split(",")
        
df1.head()

,Drug_name,US_brands
0,Antipyrine And Benzocaine (Otic Route),"[AB Otic, Aurodex, Auroto, Benzotic, Dec-A..."
1,Erythromycin (Topical Route),"[ATS, Akne-Mycin, Emcin, Emgel, Ery, Eryc..."
2,Pyrethrum Extract And Piperonyl Butoxide (Topi...,"[A200 Maximum Strength, A200 Time-Tested Form..."
3,Pyrethrum Extract And Piperonyl Butoxide (Topi...,"[A200 Maximum Strength, A200 Time-Tested Form..."
4,Laxative (Oral Route),"[A-3 Revised, Alocass, Amitiza, Benefiber, ..."


In [9]:
df1=list(df1['US_brands'])
df1[2]

['A200 Maximum Strength',
 ' A200 Time-Tested Formula',
 ' Lice-X',
 ' Licide',
 ' Medi-Lice Maximum Strength',
 ' Pronto Maximum Strength',
 ' Pyrinex',
 ' Pyrinyl',
 ' Rid',
 ' Tisit']

In [10]:
# Creating pivot table acceptable to apriori algorithm
te = TransactionEncoder()
te_ary = te.fit(df1).transform(df1)
df1 = pd.DataFrame(te_ary, columns=te.columns_)
df1.head()

,A-G Profen,A200 Time-Tested Formula,Abilify Discmelt,Abilify Maintena,Accutane,Aciphex Sprinkle,Acne 10 Gel,Acne Wash,Actamin Maximum Strength,Actiq,Actoplus Met XR,Acular LS,Acular PF,Acuvail,Adderall XR,Adenoscan,Adipex-P,Adoxa CK,Adoxa Pak,Adoxa TT,Adrenalin,Adrenalin Chloride,Advair Diskus 10050,Advair Diskus 25050,Advair Diskus 50050,Advair HFA,Advair HFA 11521,Advair HFA 23021,Advair HFA 4521,Advil,Adynovate,Aerobid-M,Aerospan,Afeditab CR,Afinitor Disperz,Afrin,Aftate,Ak-Dilate,Akne-Mycin,Ala-Hist DHC,...,Zepatier,Zerbaxa,Zerit,Zerviate,Zetia,Zevalin In-111,Ziac,Ziagen,Zinacef,Zinbryta,Zingo,Zinplava,Zioptan,Zirabev,Zirgan,Zithromax,Zocor,Zofran,Zoladex,Zolgensma,Zolinza,Zoloft,Zomig,Zonegran,Zontivity,Zostavax,Zosyn,Zovirax,Zulresso,Zurampic,ZyPREXA,ZyPREXA IntraMuscular,Zydelig,Zyflo,Zykadia,Zylet,Zyloprim,Zymar,ZyrTEC-D,Zyvox
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [11]:
def convert_into_binary(x):
    if x > 0:
        return 1
    else:
        return 0
    
df1 = df1.applymap(convert_into_binary)

Providing data to apriori algorithm and getting itemsets i.e. manufacturers

In [12]:
frequent_itemsets = apriori(df1, min_support=0.02, use_colnames=True)

In [13]:
frequent_itemsets

,support,itemsets
0,0.022433,( Aldex D)
1,0.026057,( Alka-Seltzer Plus Cold and Sinus)
2,0.022433,( AllanVan-S)
3,0.022433,( Allegra-D)
4,0.028645,( Aller-Chlor)
...,...,...
131118,0.022433,"( Pediatex 12D, ZyrTEC-D, Deconamine SR, Se..."
131119,0.022433,"( Pediatex 12D, ZyrTEC-D, Deconamine SR, Se..."
131120,0.022433,"( Pediatex 12D, ZyrTEC-D, Deconamine SR, Se..."
131121,0.022433,"( Pediatex 12D, ZyrTEC-D, Deconamine SR, Se..."


In [15]:
# We have association rules which need to put on frequent itemset
# setting threshold with lift 
rules_mlxtend = association_rules(frequent_itemsets, metric = 'support', min_threshold=0.025)
rules_mlxtend.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,( Genapap Sinus),( Alka-Seltzer Plus Cold and Sinus),0.028818,0.026057,0.026057,0.904192,34.700599,0.025306,10.165531
1,( Alka-Seltzer Plus Cold and Sinus),( Genapap Sinus),0.026057,0.028818,0.026057,1.000000,34.700599,0.025306,inf
2,( Tylenol),( Alka-Seltzer Plus Cold and Sinus),0.032097,0.026057,0.026057,0.811828,31.155914,0.025221,5.175812
3,( Alka-Seltzer Plus Cold and Sinus),( Tylenol),0.026057,0.032097,0.026057,1.000000,31.155914,0.025221,inf
4,( Aller-Chlor),( Benadryl),0.028645,0.028645,0.028645,1.000000,34.909639,0.027825,inf


In [17]:
# Get maximum confidence and lift
print(rules_mlxtend['confidence'][rules_mlxtend['confidence']==max(rules_mlxtend['confidence'])])

print(rules_mlxtend['lift'][rules_mlxtend['lift']==max(rules_mlxtend['lift'])])

1      1.0
3      1.0
4      1.0
5      1.0
7      1.0
      ... 
185    1.0
186    1.0
188    1.0
189    1.0
190    1.0
Name: confidence, Length: 156, dtype: float64
26    38.377483
31    38.377483
Name: lift, dtype: float64


In [18]:
#Convert frozenset objects into string
rules_mlxtend["antecedents"] = rules_mlxtend["antecedents"].apply(lambda x: ', '.join(list(x))).astype("unicode")
rules_mlxtend["consequents"] = rules_mlxtend["consequents"].apply(lambda x: ', '.join(list(x))).astype("unicode")

In [19]:
df1.columns

Index([' A-G Profen', ' A200 Time-Tested Formula', ' Abilify Discmelt',
       ' Abilify Maintena', ' Accutane', ' Aciphex Sprinkle', ' Acne 10 Gel',
       ' Acne Wash', ' Actamin Maximum Strength', ' Actiq',
       ...
       'ZyPREXA', 'ZyPREXA IntraMuscular', 'Zydelig', 'Zyflo', 'Zykadia',
       'Zylet', 'Zyloprim', 'Zymar', 'ZyrTEC-D', 'Zyvox'],
      dtype='object', length=4562)

In [21]:
medicines = [' A-G Profen', ' A200 Time-Tested Formula', ' Abilify Discmelt',
       ' Abilify Maintena', ' Accutane', ' Aciphex Sprinkle', ' Acne 10 Gel',
       ' Acne Wash', ' Actamin Maximum Strength', ' Actiq',
       'ZyPREXA', 'ZyPREXA IntraMuscular', 'Zydelig', 'Zyflo', 'Zykadia',
       'Zylet', 'Zyloprim', 'Zymar', 'ZyrTEC-D', 'Zyvox']

In [29]:
prescribed_item = " Genapap Sinus"


recommended_item = rules_mlxtend['consequents'].loc[(rules_mlxtend["antecedents"] == prescribed_item) & rules_mlxtend['lift'] >=1]
recommended_item = str(recommended_item).split("Name")[0].replace(" ","").replace("\n","")[1:]

print("You Have prescribed",bought_item,"\nYou can suggest some more -> ",recommended_item," !!!")

You Have prescribed  Genapap Sinus 
You can suggest some more ->  Alka-SeltzerPlusColdandSinus19Tylenol30Tylenol,Alka-SeltzerPlusColdandSinus  !!!


Conclusion : Doctors abraod can get help from this to suggest different manufacturers  (brands of medicines) available for precribed medicine.